In [6]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import ipywidgets as widgets

# The Biological Neuron
- Neuron can be considered to be the simplest processing unit of brain as well as an information highway
- Biological neuron has dendrites, soma and axons
- Dendrites are tree like processes which receive inputs from other neurons or bio-transducers (sensors)
- Dendrites contain recceptors for neurotransmitters, which activate ligand-gated channels and eventualy the voltage-gated channels
- A flux of ions is created and depending on the intracellular potential obtained, a neuron may or may not end up generating an acion potential
- Cell body or soma contains the cell nucleus and regulates the vital functions of the cell
- Axons enable propagation of action potential (saltatory for PNS and white matter of CNS) and employ a transport chain to transfer proteins to the distal end
- At the distal end there is an axon terminal which synapses with the next neuron
![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Blausen_0657_MultipolarNeuron.png/800px-Blausen_0657_MultipolarNeuron.png)

In [7]:
class HHModel:

    class Gate:
        alpha, beta, state = 0, 0, 0
 
        def update(self, deltaTms):
            alphaState = self.alpha * (1-self.state)
            betaState = self.beta * self.state
            self.state += deltaTms * (alphaState - betaState)
 
        def setInfiniteState(self):
            self.state = self.alpha / (self.alpha + self.beta)
 
    m, n, h = Gate(), Gate(), Gate()
    Cm = 1
 
    def __init__(self, ENa, EK, EKleak, gNa, gK, gKleak, startingVoltage=0):
        self.ENa = ENa
        self.EK  = EK
        self.EKleak = EKleak
        self.gNa = gNa
        self.gK = gK
        self.gKleak =  gKleak
        self.Vm = startingVoltage
        self.UpdateGateTimeConstants(startingVoltage)
        self.m.setInfiniteState()
        self.n.setInfiniteState()
        self.h.setInfiniteState()

    def UpdateGateTimeConstants(self, Vm):
        self.n.alpha = .01 * ((10-Vm) / (np.exp((10-Vm)/10)-1))
        self.n.beta = .125*np.exp(-Vm/80)
        self.m.alpha = .1*((25-Vm) / (np.exp((25-Vm)/10)-1))
        self.m.beta = 4*np.exp(-Vm/18)
        self.h.alpha = .07*np.exp(-Vm/20)
        self.h.beta = 1/(np.exp((30-Vm)/10)+1)

    def UpdateCellVoltage(self, stimulusCurrent, deltaTms):
        self.INa = np.power(self.m.state, 3) * self.gNa * \
            self.h.state*(self.Vm-self.ENa)
        self.IK = np.power(self.n.state, 4) * self.gK * (self.Vm-self.EK)
        self.IKleak = self.gKleak * (self.Vm-self.EKleak)
        Isum = stimulusCurrent - self.INa - self.IK - self.IKleak
        self.Vm += deltaTms * Isum / self.Cm

    def UpdateGateStates(self, deltaTms):
        self.n.update(deltaTms)
        self.m.update(deltaTms)
        self.h.update(deltaTms)
 
    def Iterate(self, stimulusCurrent=0, deltaTms=0.05):
        self.UpdateGateTimeConstants(self.Vm)
        self.UpdateCellVoltage(stimulusCurrent, deltaTms)
        self.UpdateGateStates(deltaTms)

# The Hodgkin-Huxley Model
- Modelling a simplified biological neuron involves tracking the flux of ions through 3 ion channels - Sodium, Potassium and Leak channels.
- Changes in ionic concentration of intracellular and extracellular environment has two effects
- Primarily, it induces an action potential when the ICF potential exceeds the threshold limit
- It also controls the state of voltage controlled gates (Na and K, in this case)
- In this project we have modelled these events as differential equations and plotted their evolution over a finite time
![](https://raw.githubusercontent.com/swharden/HHSharp/master/dev/theory.png)


In [8]:
class Simulation:

    def __init__(self, model):
        self.model = model
        self.CreateArrays(0, 0)
        pass

    def CreateArrays(self, pointCount, deltaTms):
        self.times = np.arange(pointCount) * deltaTms
        self.Vm = np.empty(pointCount)
        self.INa = np.empty(pointCount)
        self.IK = np.empty(pointCount)
        self.IKleak = np.empty(pointCount)
        self.StateN = np.empty(pointCount)
        self.StateM = np.empty(pointCount)
        self.StateH = np.empty(pointCount)

    def Run(self, stimulusWaveform, stepSizeMs):
        if (stepSizeMs > 0.05):
            warnings.warn("step sizes < 0.05 ms are recommended")
        assert isinstance(stimulusWaveform, np.ndarray)
        self.CreateArrays(len(stimulusWaveform), stepSizeMs)
        print(f"-----\nsimulating {len(stimulusWaveform)} time points...")
        for i in range(len(stimulusWaveform)):
            self.model.Iterate(stimulusWaveform[i], stepSizeMs)
            self.Vm[i] = self.model.Vm
            self.INa[i] = self.model.INa
            self.IK[i] = self.model.IK
            self.IKleak[i] = self.model.IKleak
            self.StateH[i] = self.model.h.state
            self.StateM[i] = self.model.m.state
            self.StateN[i] = self.model.n.state
        print("simulation complete\n-----")

# External Stimulation

- A hodgkin-huxley neuron is a stable entity which has a resting membrane potential below that of the threshold potential
- Without environmental stimulation, a neuron would continue to exist as is
- Here we are providing a square wave as an external potential whose intensity, duration can be configured beloy
![](https://www.researchgate.net/profile/Dylan-Shepardson/publication/43027348/figure/fig3/AS:669570754961420@1536649516718/Equivalent-circuit-for-Hodgkin-Huxley-model-of-the-squid-giant-axon-from-16-R-N-a.ppm)

In [9]:
def HHplot(ENa, EK, EKleak, gNa, gK, gKleak, stepNum, stepSize, stimStart, stimEnd, stimIntensity):
    model = HHModel(ENa, EK, EKleak, gNa, gK, gKleak)
 
    stim = np.zeros(int(stepNum))
    stim[stimStart:stimEnd] = stimIntensity
 
    sim = Simulation(model)
    sim.Run(stimulusWaveform=stim, stepSizeMs=stepSize)
    plt.figure(figsize=(10, 8))
 
    ax1 = plt.subplot(411)
    ax1.plot(sim.times, sim.Vm - 70, color='b')
    ax1.set_ylabel("Potential (mV)")
    ax1.set_title("Hodgkin-Huxley Spiking Neuron Model", fontSize=16)
 
    ax2 = plt.subplot(412)
    ax2.plot(sim.times, stim, color='r')
    ax2.set_ylabel("Stimulation (µA/cm²)")
 
    ax3 = plt.subplot(413, sharex=ax1)
    ax3.plot(sim.times, sim.StateH, label='h')
    ax3.plot(sim.times, sim.StateM, label='m')
    ax3.plot(sim.times, sim.StateN, label='n')
    ax3.set_ylabel("Activation (frac)")
    ax3.legend()
 
    ax4 = plt.subplot(414, sharex=ax1)
    ax4.plot(sim.times, sim.INa, label='VGSC')
    ax4.plot(sim.times, sim.IK, label='VGKC')
    ax4.plot(sim.times, sim.IKleak, label='KLeak')
    ax4.set_ylabel("Current (µA/cm²)")
    ax4.set_xlabel("Simulation Time (milliseconds)")
    ax4.legend()
 
    plt.tight_layout()
    plt.show()

#Plot with the Default Values
- You can configure the values of ENa, EK and ELeak and gNa, gK and gLeak
- The step number and size can be changed
- The simulation runs for a length of (Step Number * Step Size) where step size is calculated in milliseconds
- For the experiment an external stimulus is being given, you can configure the starting step, ending step and intensity of the stimulus
- To figure out the starting step, simply divide the starting time you want by the chosen step size

#### Kindly connect to a runtime and run all the preceding cells (or hit **Run All** from Runtime menu)

In [10]:
%matplotlib inline
style = {'description_width': 'initial'}
 
ENa_widget = widgets.FloatSlider(min=0, max=200, step=0.5, value=115, description="ENa")
EK_widget = widgets.FloatSlider(min=-30, max=0, step=0.5, value=-12, description="EK")
EKleak_widget = widgets.FloatSlider(min=0, max=20, step=0.5, value=10.6, description="EKleak")
 
gNa_widget = widgets.FloatSlider(min=0, max=200, step=0.5, value=120, description="gNa")
gK_widget = widgets.FloatSlider(min=0, max=50, step=0.5, value=36, description="gK")
gKleak_widget = widgets.FloatSlider(min=0, max=5, step=0.01, value=0.3, description="gKleak")
 
stepNum_widget = widgets.IntSlider(min=0, max=50000, step=1000, value=20000, description="Step Number")
stepSize_widget = widgets.FloatSlider(min=0, max=0.1, step=0.005, value=0.01, description="Step Size")
 
stimStart_widget = widgets.IntSlider(min=0, max=50000, step=1000, value=7000, description="Stimulus Start", style=style)
stimEnd_widget = widgets.IntSlider(min=0, max=50000, step=1000, value=13000, description="Stimulus End", style =style)
stimIntensity_widget = widgets.IntSlider(min=0, max=100, step=1, value=50, description="Stimulus Intensity", style =style)
 
control = widgets.VBox([ENa_widget, EK_widget, EKleak_widget, gNa_widget, gK_widget, gKleak_widget, stepNum_widget, stepSize_widget, stimStart_widget, stimEnd_widget, stimIntensity_widget])

plot = widgets.interactive_output(HHplot, {'ENa':ENa_widget, 'EK':EK_widget, 'EKleak':EKleak_widget, 'gNa':gNa_widget, 'gK':gK_widget, 'gKleak':gKleak_widget, 'stepNum':stepNum_widget, 'stepSize':stepSize_widget, 'stimStart':stimStart_widget, "stimEnd":stimEnd_widget, "stimIntensity":stimIntensity_widget})
display(control, plot)

Output()

# Sources and Acknowledgement


1. [Fundamentals of Computational Neuroscience by Thomas T. Trappenberg](https://books.google.co.in/books/about/Fundamentals_of_Computational_Neuroscien.html?id=4PDsA1EVCx0C&redir_esc=y)
2. [Project Encephalon](https://projectencephalon.org)
3. [SWHarden on Github](https://github.com/swharden)
4. [Wikimedia Commons](https://commons.wikimedia.org/wiki/Main_Page)
5. [Dylan Sheperdson on Research Gate](https://www.researchgate.net/figure/Equivalent-circuit-for-Hodgkin-Huxley-model-of-the-squid-giant-axon-from-16-R-N-a_fig3_43027348)
6. [Samhita Alla on Towards Data Science](https://towardsdatascience.com/a-brief-introduction-to-computational-neuroscience-part-1-42171791f613)

